In [25]:
import requests
import json
import pandas as pd
import time
import re
import urllib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import boto3
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:.2f}'.format

In [26]:
s3 = boto3.client('s3')

In [27]:
from datetime import datetime
def write_to_s3(df, file_path, bucket_name='incioman-data-analysis'):
    print(f"Uploading {file_path} to {datetime.today().date().strftime('%Y%m%d')} on s3")
    json_string = df.to_json(orient='records')
    s3.put_object(Body=json_string, Bucket=bucket_name, Key=file_path)

In [28]:
def load_from_s3(file_path_in_bucket, bucket_name='incioman-data-analysis'):
    print(f"Trying to load swaps from {bucket_name} {file_path_in_bucket}")
    # Load the CSV file from S3
    obj = s3.get_object(Bucket=bucket_name, Key=file_path_in_bucket)
    df = pd.read_csv(obj['Body'])
    print(f"Loaded swaps from {bucket_name} {file_path_in_bucket}")
    return df

In [29]:
file_path_in_bucket = 'astro_trades/data/raw_swaps/swaps.csv'
try:
    swaps_df = load_from_s3(file_path_in_bucket)
    swaps_df.timestamp = pd.to_datetime(swaps_df.timestamp, format='mixed')
except Exception as e:
    print(f"No file yet {e}")
    swaps_df = None

Trying to load swaps from incioman-data-analysis astro_trades/data/raw_swaps/swaps.csv
Loaded swaps from incioman-data-analysis astro_trades/data/raw_swaps/swaps.csv


In [30]:
file_path_in_bucket = 'astro_trades/data/balances/20230720.csv'
try:
    balances_df = load_from_s3(file_path_in_bucket)
    balances_df['traderAddress'] = balances_df['address']
except Exception as e:
    print(f"No file yet {e}")
    balances_df = None

Trying to load swaps from incioman-data-analysis astro_trades/data/balances/20230720.csv
Loaded swaps from incioman-data-analysis astro_trades/data/balances/20230720.csv


In [31]:
_sell = swaps_df[swaps_df.offerAsset=='ASTRO']
_buy = swaps_df[swaps_df.returnAsset=='ASTRO']
_sell['astro_amount'] =  - _sell.offerAmount
_buy['astro_amount'] =  _buy.returnAmount
swaps_df = pd.concat([_sell, _buy])

/tmp/ipykernel_310/142230925.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _sell['astro_amount'] =  - _sell.offerAmount
/tmp/ipykernel_310/142230925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _buy['astro_amount'] =  _buy.returnAmount


tokens = [
"xASTRO",
"SAYVE",
"ORNE",
"ASTRO",
"ampLUNA",
"ROAR",
"boneLuna"
]

import requests
import json

def get_balances(address):
    url = "https://starscream-terra-mainnet.chainscope.dev/"
    headers = {
        "Content-Type": "application/json"
    }
    
    body = {
        "query": """
        query Query($address: String!, $tokenContracts: [String!]!) {
            tokenBalances(address: $address, tokenContracts: $tokenContracts)
        }
        """,
        "variables": {
            "address": f"{address}",
            "tokenContracts": [
                "terra1x62mjnme4y0rdnag3r8rfgjuutsqlkkyuh4ndgex0wl3wue25uksau39q8",
                "terra1xp9hrhthzddnl7j5du83gqqr4wmdjm5t0guzg9jp6jwrtpukwfjsjgy4f3",
                "terra19p20mfnvwh9yvyr7aus3a6z6g6uk28fv4jhx9kmnc2m7krg27q2qkfenjw",
                "terra1nsuqsk6kh58ulczatwev87ttq2z6r3pusulg9r24mfj2fvtzd4uq3exn26",
                "terra1ecgazyd0waaj3g7l9cmy5gulhxkps2gmxu9ghducvuypjq68mq2s5lvsct",
                "terra1lxx40s29qvkrcj8fsa3yzyehy7w50umdvvnls2r830rys6lu2zns63eelv",
                "terra17aj4ty4sz4yhgm08na8drc0v03v2jwr3waxcqrwhajj729zhl7zqnpc0ml"
            ]
        },
        "operationName": "Query"
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(body))
    
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        print("Request failed with status code:", response.status_code)

addresses = set(df.traderAddress.tolist())

balances = []
for address in addresses:
    try:
        balance_dict = {k: int(v)/1000000 for k, v in zip(tokens, get_balances(address)['data']['tokenBalances'])}
        balance_dict['traderAddress']= address
        balances.append(balance_dict)
    except:
        print(f"Error processing: {address}")

balances_df = pd.DataFrame(balances)
balances_df['total_astro_holdings'] = balances_df['ASTRO'] + balances_df['xASTRO']

df_ = pd.DataFrame(balances_df).merge(df, on='traderAddress')

balances_df.total_astro.sort_values().cumsum().reset_index(drop=True).plot(kind='line')

In [33]:
percentile = 0.99

In [34]:
astro_price = 0.022

In [35]:
print(f"Amount of ASTRO that defines a whale ({percentile} percentile): {balances_df.total_astro.quantile(percentile)}")

Amount of ASTRO that defines a whale (0.99 percentile): 117915.78052971986


In [36]:
whales = balances_df[balances_df.total_astro > balances_df.total_astro.quantile(percentile)]

In [37]:
print(f"How many whales: {len(whales)}")

How many whales: 200


In [38]:
daily_delta = swaps_df[['traderAddress','date','astro_amount']].groupby(['traderAddress','date']).sum().reset_index()

In [39]:
daily_delta[["traderAddress","astro_amount"]]\
    .groupby(['traderAddress']).sum().reset_index()\
    .sort_values(by='astro_amount').head()

,traderAddress,astro_amount
45,terra129atwlwg5dwuypmce92gz7y5rwzdyxlwd8ne9r,-1440087.71
123,terra147vzdxzn5tq4fm7z6f5q8x46xfha7e34w4sy9j,-1402786.24
498,terra1evtkrqahmavh4t5x6squzpwfcr4nl5ssj67edz,-1222528.00
811,terra1pgvewpk8kd30f2zm85sn69k9cgkp3j8szq00j7,-785389.82
657,terra1jz2wmfvcudvkpdckzxjemy62852x5e4huf679q,-757315.20


In [40]:
whales_daily = whales.merge(daily_delta[daily_delta.date>='2023-07-01'],on='traderAddress', how='left').fillna(0)

In [46]:
tt = whales_daily[whales_daily.date != 0][['date','astro_amount']]\
        .groupby('date')\
        .sum()\
        .reset_index()\
        .sort_values(by='date')

In [47]:
write_to_s3(tt, file_path = f"astro_trades/data/whales_daily_delta/{datetime.today().date().strftime('%Y%m%d')}.json")

Uploading astro_trades/data/whales_daily_delta/20230731.json to 20230731 on s3


In [22]:
tt = whales_daily[whales_daily.date!=0][['address','date','astro_amount']]\
                    .groupby(['address','date'])\
                    .sum().reset_index().sort_values(by='date')
write_to_s3(tt, file_path = f"astro_trades/data/whales_daily_delta_by_address/{datetime.today().date().strftime('%Y%m%d')}.json")

Uploading astro_trades/data/whales_daily_delta_by_address/20230728.json to 20230728 on s3
